# Open Street Map Data Analysis

In this project, the ATMs and branches of the banks in Ankara, capital of Turkey are analyzed and inserted into SQL database.

In order to create a database of branches and ATMs of banks in Ankara, three csv files need to be created which are banks.csv, branches.csv and atms.csv representing banks table in SQL, branches of banks table in SQL and ATMS of banks table in SQL, respectively. 

## Map Area

The map is my hometown, so I am more interested in finding ATMs and Banks in the city.

The osm file size is 281.3 MB which can be downloaded here : https://s3.amazonaws.com/metro-extracts.mapzen.com/ankara_turkey.osm.bz2


## Problems Encountered in the Map

According to the documantation in openstreetmap.org, the banks and atms exist in the data as amenities. The names of the banks can be different so that first of all the names of the banks are analyzed. 

The brances of banks are represented as amenity=bank in the dataset and ATMs are represented as amenity=atm.

In order to analyze the problems in the dataset, I checked the distinct bank names.



In [126]:
import xml.etree.cElementTree as ET
import pprint
import re

OSMFILE = "ankara_turkey.osm"



In [149]:
def is_bank_branch(elem):
    return (elem.attrib['k'] == "amenity" and elem.attrib['v'] == "bank")

def is_bank_atm(elem):
    return (elem.attrib['k'] == "amenity" and elem.attrib['v'] == "atm")

bank_names = set()
def distinct_bank_names(osm_file):
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node":
            name = ""
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == "name":
                    name = tag.attrib['v']
                if is_bank_branch(tag) or is_bank_atm(tag):
                    bank_names.add(name.lower())
    


distinct_bank_names(OSMFILE)
for name in bank_names:
    print name


deniz bank
ingbank atm
ziraat bankasi
vakıfbank (atm)
finansbank pursaklar
ptt
akbank atm
akbank
bank asya
vakıf bank
vakıfbank söğütözü şubesi
tekstilbank
finansbank atm
halkbank
türkiye iş bankası atm
qnb finansbank
şekerbank
albaraka türk
kuveyt türk
turkiye is bankasi
işbankası atm
tc ziraat bankasi
teb atm
odeabank
vakıfbank
halkbankası atm
albaraka
işbankası (atm)
abank
halk banksı
halkbank hüseyingazi şubesi
teb
cüzdanmatik
emniyet md. ek bina
akbank, tr-ankara
yapı kredi leasing
türkiye finans katılım bankası
ziraat bankası güneşevler şubesi
ykb atm
garanti bankası atm
yapıkredi atm
iş b.
garanti bankası
yapı kredi bankası atm
t.c. ziraat bankası a.ş.
fibabanka
garanti
türkiye finans katılım
burgan bank
finans bank
vakıfbank atm
ing bank atm
a bank
ziraat b.
yapı kredi bankası
finansbank
halkbankası (atm)
yapi kredi
yapı kredi bankası çukurambar şubesi
ing bankası
hsbc atm
yapı kredi
hsbc pursaklar
t-bank
garanti bankası (atm)
ziraat bankası atm
nakit yükleme noktası
vakıfbank

When the names of the banks were analyzed, some problems are clearly seen in the dataset.

- Some bank names are written in different formats. For example, Akbank is written in the forms of "AKBANK","Ak Bank", "Akbank", "Akbank - Ya\u015famkent \u015eubesi". Those strings needed to be fixed to create a banks table in SQL which includes unique banks.

- ATMs of banks are stated using "ATM" or "(ATM)" strings at the end of the bank names.  

- Some of the banks changed their name over time, for example "Finansbank" is now "QNB Finansbank". Those changes need to be considered.

- There are some names that do not represent a bank name, for example "PTT" which is national post and telegraph directorate in Turkey. Those need to be exclueded.

- Abbreviated names are another problem, for example YKB for Yapi Kredi Bankasi.

- Since the names of banks are in Turkish, some encoding problems occur.


In order to solve those problems, I have created a expected bank names list which are all unicode and lower case. The names which are not included in the expected bank names list are updated using a dictionary which converts the problematic bank names to expected names.



In [339]:
#RegEx for getting the first part of the string
# encoding=utf-8
expected_bank_names = ["denizbank", "ingbank", "ziraat bankası", "vakıfbank", "qnb finansbank", 
                       "akbank", "bank asya", "tekstilbank", "halkbank", "iş bankası", "şekerbank",
                       "albaraka türk", "kuveyt türk", "teb", "odeabank", "abank", "türkiye finans katılım bankası",
                       "garanti bankası", "yapı kredi", "fibabanka", "burgan bank", "hsbc", "t-bank", "anadolubank"]

bank_name_re = re.compile(r'\S+\b\.?$', re.IGNORECASE)
bank_name_mapping = {
    "deniz bank" : "denizbank",
    "ingbank atm" : "ingbank",
    "ziraat bankasi" : "ziraat bankasi",
    "vakıfbank (atm)" : "vakifbank",
    "finansbank pursaklar" : "qnb finansbank",
    "akbank atm" : "akbank",
    "vakıf bank" : "vakıfbank",
    "vakıfbank söğütözü şubesi": "vakıfbank",
    "finansbank atm" : "qnb finansbank",
    "türkiye iş bankası atm" : "iş bankası",
    "turkiye is bankasi" : "iş bankası",
    "işbankası atm" : "iş bankası",
    "tc ziraat bankasi" : "ziraat bankası",
    "teb atm" : "teb",
    "halkbankası atm" : "halkbank",
    "albaraka" : "albaraka türk",
    "işbankası (atm)" : "iş bankası",
    "halk banksı" : "halkbank",
    "halkbank hüseyingazi şubesi" : "halkbank",
    "akbank, tr-ankara" : "akbank",
    "yapı kredi leasing" : "yapı kredi",
    "türkiye finans katılım bankası" : "qnb finansbank",
    "ziraat bankası güneşevler şubesi" : "ziraat bankası",
    "ykb atm" : "yapı kredi",
    "garanti bankası atm" : "garanti bankası",
    "yapıkredi atm" : "yapı kredi",
    "iş b." : "iş bankası",
    "yapı kredi bankası atm" :"yapı kredi",
    "t.c. ziraat bankası a.ş." : "ziraat bankası",
    "garanti" : "garanti bankası",
    "türkiye finans katılım" : "qnb finansbank",
    "finans bank" : "qnb finansbank",
    "vakıfbank atm": "vakıfbank",
    "ing bank atm" : "ingbank",
    "a bank" : "abank",
    "ziraat b." : "ziraat bankası",
    "yapı kredi bankası" : "yapı kredi",
    "finansbank" : "qnb finansbank",
    "halkbankası (atm)" : "halkbank",
    "yapı kredi bankası çukurambar şubesi" : "yapı kredi",
    "ing bankası" : "ingbank",
    "hsbc atm" : "hsbc",
    "hsbc pursaklar": "hsbc",
    "garanti bankası (atm)" : "garanti bankası",
    "ziraat bankası atm" : "ziraat bankası",
    "vakıfbank tpao şubesi": "vakıfbank",
    "akbank - yaşamkent şubesi" :  "akbank",
    "yapıkredi bankası" : "yapı kredi",
    "garanti atm" : "garanti bankası",
    "kuveyt türk atm" : "kuveyt türk",
    "ing atm" : "ingbank",
    "finansbank (atm)" : "qnb finansbank",
    "ing bank" : "ingbank",
    "akbank (atm)" :  "akbank",
    "iş bankası" : "iş bankası",
    "garanti bank" : "garanti bankası",
    "denizbank atm" : "denizbank",
    "vakifbank": "vakıfbank",
    "yapıkredi" : "yapı kredi",
    "ziraat atm" : "ziraat bankası",
    "garanti bankası - yaşmkent şubesi" : "garanti bankası",
    "ak bank atm" :  "akbank",
    "ziraat bankası pursaklar" : "ziraat bankası",
    "halkbank atm" : "halkbank",
    "ziraatbank atm" : "ziraat bankası",
    "iş bankası atm" : "iş bankası",
    "garanti bankasi" : "garanti bankası",
    "türkiye iş bankası" : "iş bankası",
    "asya bank" : "bank asya",
    "denizbank": "denizbank",
    "yapi kredi" : "yapı kredi",
    "garanti":"garanti bankası"

}

def convert_unicode_if_not(strn):
    if strn == None:
        return None
    elif isinstance(strn, unicode):
        return strn
    else:
        return unicode(strn, 'utf-8')
    
def update_bank_name(name):
    converted_name = convert_unicode_if_not(name).lower()
    if converted_name in [convert_unicode_if_not(x).lower() for x in expected_bank_names]:
        return converted_name
    elif converted_name in [x.decode('utf-8') for x in bank_name_mapping.keys()]:
        return bank_name_mapping[converted_name.encode('utf-8')]



### Converting bank names to unicode for matching

There are both unicode and non-unicode strings in the dataset. Therefore the non-unicode strings need to be converted and unicode strings shall remain the same.



```
def convert_unicode_if_not(strn):
    if strn == None:
        return None
    elif isinstance(strn, unicode):
        return strn
    else:
        return unicode(strn, 'utf-8')
```

### Updating bank names 

Bank names are converted to proper names of the banks using the expected bank names list.


```
def update_bank_name(name):
    converted_name = convert_unicode_if_not(name).lower()
    if converted_name in [convert_unicode_if_not(x).lower() for x in expected_bank_names]:
        return converted_name
    elif converted_name in [x.decode('utf-8') for x in bank_name_mapping.keys()]:
        return bank_name_mapping[converted_name.encode('utf-8')]

```

## Creating csv files


After developing converting mechanism, the csv files are generated to seed the data into SQL database.


In [365]:
import csv

def create_csv(arr, headers, path):
    with open(path, 'wb') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = headers, delimiter = ',')
        writer.writeheader()
        for el in arr:
            writer.writerow(el)
            
            

BANKS_PATH = "banks.csv"
BRANCHES_PATH = "branches.csv"
ATMS_PATH = "atms.csv"

atms = {}

BANKS_FIELDS = ['id', 'name']
BRANCH_FIELDS = ['id', 'bank_id', 'latitude', 'longitude', 'changeset', 'uid', 'timestamp', 'version']
ATM_FIELDS = ['id', 'bank_id', 'latitude', 'longitude', 'changeset', 'uid', 'timestamp', 'version']

# BANKS
banks = list()

def fill_banks():
    for idx, name in enumerate(expected_bank_names):
        bank = {}
        bank['name'] = name
        bank['id'] = idx
        banks.append(bank)
        
        
def find_bank_id_by_name(name):
    for bank in banks:
        if bank['name'].decode("utf-8") == convert_unicode_if_not(name):
            return bank['id']
    return None

fill_banks()          
create_csv(banks, BANKS_FIELDS, BANKS_PATH)



In [366]:
# BRANCHES
branches = list()

def fill_branches(osm_file):
    idx = 0
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node":
            branch = {}
            name = ""
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == "name":
                    name = tag.attrib['v']
                if is_bank_branch(tag):
                    if find_bank_id_by_name(update_bank_name(name)) == None:
                        continue
                    branch['id'] = idx
                    branch['bank_id'] = find_bank_id_by_name(update_bank_name(name))
                    branch['latitude'] = elem.attrib['lat']
                    branch['longitude'] = elem.attrib['lon']
                    branch['version'] = elem.attrib['version']
                    branch['timestamp'] = elem.attrib['timestamp']
                    branch['changeset'] = elem.attrib['changeset']
                    branch['uid'] = elem.attrib['uid']
                    branches.append(branch)
                    idx += 1

fill_branches(OSMFILE)
create_csv(branches, BRANCH_FIELDS, BRANCHES_PATH)

In [367]:
# ATMS
atms = list()

def fill_atms(osm_file):
    idx = 0
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node":
            atm = {}
            name = ""
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == "name":
                    name = tag.attrib['v']
                if is_bank_atm(tag):
                    if find_bank_id_by_name(update_bank_name(name)) == None:
                        continue
                    atm['id'] = idx
                    atm['bank_id'] = find_bank_id_by_name(update_bank_name(name))
                    atm['latitude'] = elem.attrib['lat']
                    atm['longitude'] = elem.attrib['lon']
                    atm['version'] = elem.attrib['version']
                    atm['timestamp'] = elem.attrib['timestamp']
                    atm['changeset'] = elem.attrib['changeset']
                    atm['uid'] = elem.attrib['uid']
                    atms.append(atm)
                    idx += 1

fill_atms(OSMFILE)
create_csv(atms, ATM_FIELDS, ATMS_PATH)

## Seeding to SQL

### Creating tables


In [370]:
import sqlite3

banks_table_create_q = " CREATE TABLE banks ( id INTEGER PRIMARY KEY NOT NULL, name TEXT);"
branches_table_create_q = "CREATE TABLE branches (id INTEGER PRIMARY KEY NOT NULL, bank_id INTEGER, latitude REAL, longitude REAL, uid INTEGER, version INTEGER, changeset INTEGER, timestamp TEXT, FOREIGN KEY (bank_id) REFERENCES banks(id));"
atms_table_create_q = "CREATE TABLE atms (id INTEGER PRIMARY KEY NOT NULL, bank_id INTEGER, latitude REAL, longitude REAL, uid INTEGER, version INTEGER, changeset INTEGER, timestamp TEXT, FOREIGN KEY (bank_id) REFERENCES banks(id));"

con = sqlite3.connect("osm_bank_locations.db")
cur = con.cursor()
cur.execute(banks_table_create_q) 
cur.execute(branches_table_create_q) 
cur.execute(atms_table_create_q)



### Seeding the CSVs to DB


In [371]:
import pandas

con.text_factory = lambda x: unicode(x, 'utf-8', 'ignore')

df = pandas.read_csv(BANKS_PATH)
df.to_sql("banks", con, if_exists='append', index=False)

df = pandas.read_csv(BRANCHES_PATH)
df.to_sql("branches", con, if_exists='append', index=False)

df = pandas.read_csv(ATMS_PATH)
df.to_sql("atms", con, if_exists='append', index=False)

## Queries to SQL db


### Number of Banks, ATMs and Branches



In [378]:

count_cursor = con.execute("Select count(*) from banks;")

for row in count_cursor:
    print "Number of banks : " + str(row)
    
count_cursor = con.execute("Select count(*) from atms;")

for row in count_cursor:
    print "Number of atms : " + str(row)  
    
count_cursor = con.execute("Select count(*) from branches;")

for row in count_cursor:
    print "Number of branches : " + str(row) 

Number of banks : (24,)
Number of atms : (110,)
Number of branches : (338,)


### Banks with most branches and ATMs in Ankara

According to Openmaps dataset the bank with most branches is Ziraat Bankası. The banks with most ATMs is İş Bankası in Ankara.



In [400]:
count_cursor = con.execute("select banks.name as name, count(branches.id) as c from branches inner join banks on banks.id = branches.bank_id group by banks.name order by c DESC;")

for row in count_cursor:
    pprint.pprint(convert_unicode_if_not(row[0]) + " : " + str(row[1]))

u'ziraat bankas\u0131 : 49'
u'i\u015f bankas\u0131 : 41'
u'akbank : 32'
u'qnb finansbank : 31'
u'garanti bankas\u0131 : 29'
u'yap\u0131 kredi : 29'
u'vak\u0131fbank : 28'
u'halkbank : 21'
u'denizbank : 15'
u'teb : 13'
u'hsbc : 10'
u'ingbank : 9'
u'\u015fekerbank : 9'
u'kuveyt t\xfcrk : 6'
u'albaraka t\xfcrk : 4'
u'abank : 3'
u'bank asya : 2'
u'anadolubank : 1'
u'burgan bank : 1'
u'fibabanka : 1'
u'odeabank : 1'
u't-bank : 1'
u'tekstilbank : 1'
u't\xfcrkiye finans kat\u0131l\u0131m bankas\u0131 : 1'


In [401]:
count_cursor = con.execute("select banks.name as name, count(atms.id) as c from atms inner join banks on banks.id = atms.bank_id group by banks.name order by c DESC;")

for row in count_cursor:
    pprint.pprint(convert_unicode_if_not(row[0]) + " : " + str(row[1]))

u'i\u015f bankas\u0131 : 23'
u'akbank : 15'
u'halkbank : 12'
u'ziraat bankas\u0131 : 12'
u'garanti bankas\u0131 : 11'
u'qnb finansbank : 8'
u'yap\u0131 kredi : 8'
u'vak\u0131fbank : 7'
u'ingbank : 5'
u'denizbank : 4'
u'bank asya : 2'
u'hsbc : 1'
u'kuveyt t\xfcrk : 1'
u'teb : 1'


## Data Overview and Additional Ideas

This section includes basic statistics about the dataset, and some additional ideas about data in context.

### Contributer Analysis

The contributor with id 1386706 is the most contributed user for different banks. Openmaps dataset has limited contributers for banks in Ankara.



In [409]:
count_cursor = con.execute("select banks.name as bank, atms.uid as contributer_id, count(atms.id) as c from atms inner join banks on banks.id = atms.bank_id group by banks.name, atms.uid order by c DESC;")

for row in count_cursor:
    pprint.pprint(convert_unicode_if_not(row[0]) + " : " + str(row[1]) + " : " + str(row[2]) )

u'i\u015f bankas\u0131 : 1386706 : 14'
u'akbank : 1386706 : 11'
u'ziraat bankas\u0131 : 1386706 : 8'
u'garanti bankas\u0131 : 1386706 : 6'
u'halkbank : 1386706 : 6'
u'yap\u0131 kredi : 1386706 : 5'
u'qnb finansbank : 1386706 : 4'
u'denizbank : 1386706 : 3'
u'vak\u0131fbank : 1386706 : 3'
u'bank asya : 3442773 : 2'
u'garanti bankas\u0131 : 1686268 : 2'
u'garanti bankas\u0131 : 5349861 : 2'
u'halkbank : 5349861 : 2'
u'ingbank : 1386706 : 2'
u'i\u015f bankas\u0131 : 5349861 : 2'
u'qnb finansbank : 3755317 : 2'
u'ziraat bankas\u0131 : 436145 : 2'
u'akbank : 436145 : 1'
u'akbank : 689014 : 1'
u'akbank : 3755317 : 1'
u'akbank : 5349861 : 1'
u'denizbank : 5349861 : 1'
u'garanti bankas\u0131 : 689014 : 1'
u'halkbank : 689014 : 1'
u'halkbank : 3154081 : 1'
u'halkbank : 3755317 : 1'
u'halkbank : 4594899 : 1'
u'hsbc : 689014 : 1'
u'ingbank : 689014 : 1'
u'ingbank : 3755317 : 1'
u'ingbank : 5349861 : 1'
u'i\u015f bankas\u0131 : 436145 : 1'
u'i\u015f bankas\u0131 : 689014 : 1'
u'i\u015f bankas\u013

## Conclusion


This review of the data shows that the openmaps dataset for Ankara is inclomplete since there are a few records of branches and ATMs. This might caused by the limited set of contributers in this context. 

